In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [9]:
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy'],'max_features':['auto','sqrt','log2'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\abina\Anac\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
19 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\abina\Anac\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\abina\Anac\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\abina\Anac\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\abina\Anac\Lib\site-packages\sklearn\utils\_param_vali

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

In [14]:
y_pred=grid.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter{}".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100} 0.9183922682195829


In [17]:
print(cm)

[[78  7]
 [ 4 45]]


In [18]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9619447779111645

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000566,0.000904,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001190,0.001148,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.025174,0.003088,0.011541,0.002454,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.811321,0.832483,0.851527,0.961755,0.860845,0.052373,8
3,0.195676,0.019105,0.020564,0.002385,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.868752,0.851527,0.925272,0.943699,0.883103,0.044461,4
4,0.038607,0.015264,0.010901,0.001700,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.831253,0.851527,0.906166,0.905069,0.868231,0.031264,7
5,0.192468,0.006093,0.020131,0.005577,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.870047,0.851527,0.925272,0.962264,0.891250,0.045108,2
6,0.000840,0.000428,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001746,0.001996,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.027433,0.008103,0.011416,0.002727,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.870047,0.833323,0.906166,0.924528,0.872065,0.038777,6
9,0.197442,0.016920,0.019995,0.004818,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.888107,0.851527,0.925272,0.924528,0.887315,0.033826,3


In [22]:
age_input=int(input("Age:"))
Salary_input=float(input("Salary:"))
sex_male_input=int(input("Male 0 or 1:"))

Age:26
Salary:60000
Male 0 or 1:1


In [24]:
Future_Prediction=grid.predict([[age_input,Salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
